https://www.youtube.com/watch?v=FppOzcDvaDI&list=PLhhyoLH6Ijfw0TpCTVTNk42NN08H6UvNq&index=4

1. Get all bounding box predicitons on our test set. Green is GT and red is predicted.
2. Associated bbox confidence score is something like IOU score but not exactly.
3. How to define TP or FP is to see IOU, if IOU threshold is 0.5, then if a **predicted bbox** has an IOU of > 0.5 with a gt bbox, then it is TP, else it is FP.

- if IoU ≥0.5, classify the object detection as True Positive(TP)
- if Iou <0.5, then it is a wrong detection and classify it as False Positive(FP)
- When a ground truth is present in the image and model failed to detect the object, classify it as False Negative(FN).
- True Negative (TN): TN is every part of the image where we did not predict an object. This metrics is not useful for object detection, hence we ignore TN.


4. At minute 3.10, important, he sort the table of all test predictions, sort by descending confidence score.
    - precision = TP / (TP + FP)
    - recall = TP / (TP + FN)
    - In the table he tried to calculate the precision and recall per row. but imo for me its easy just calculate it column wise! like in the table there are 7 rows, with 3 tp and 4 fp, so precision is just 3/7 and recall is 3/4, dont get confused why recall is not 1 here, cause one didnt see any fn in the table, but TP + FN is also the total num of positive samples, which is the total num of gt bboxes which is 4. a bit confusing i must say. but col wise cant plot so a bit confusing
    
5. at min 7, he said avg precision for class dog.??

In [ ]:
y_true = ["positive", "negative", "positive", "negative", "positive", "positive", "positive", "negative", "positive", "negative"]

pred_scores = [0.7, 0.3, 0.5, 0.6, 0.55, 0.9, 0.75, 0.2, 0.8, 0.3]

thresholds = numpy.arange(start=0.2, stop=0.9, step=0.05)

**Important: you should follow this tutorial https://towardsdatascience.com/on-object-detection-metrics-with-worked-example-216f173ed31e and list out the table to calculate MAP, and a much better understanding ensues!**

<p align="center">
<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/metrics/computer_vision_metrics/mean_average_precision/blood_cells_map_calc.png" align="center"/></p>

<p align="center">
<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/metrics/computer_vision_metrics/mean_average_precision/blood_cells_map_calc_table.PNG" align="center"/></p>

- Let us label the ground truth bboxes above to be: $[1, 2, 3, 4, 5, 6, 7, 8, 9]$ (**rmb to label them above when I have time**) which are matched with
- the predicted bboxes $\{1: [a], 2: [b], 3: [c, d], 4: [e], 5: [f, g], 6: [h], 7: [], 8: [i, k], 9: [j], ?: [l] \}$
- Notice that gt bbox 7 has no predicted bbox and pred bbox $l$ has no match gt bbox.
- And also the IOUs are listed above in the table, for example pred bbox c and d have IOU of 47 and 42 with gt bbox 3.
- The MAP curve is parametrized by confidence threshold but TP/FP cutoff is parametrized by IOU (this is important to grasp!) unlike normal classification where for eg AUROC curve is parametrized by probability threshold and TP/FP are also parametrized by probablity threshold.

**Some Rules**

- We establish the IOU threshold to be 0.5, this will determine if a bbox is TP or FP.
- We establish the case of gt bbox 3, both of them are FP since they are below IOU threshold 0.5;
- We establish the case of gt bbox 5, where we take pred bbox g to be TP but pred bbox f to be FP. 
    - The source: Some detectors can output multiple detections overlapping a single ground truth. For those cases the detection with the highest confidence is considered a TP and the others are considered as FP, as applied by the PASCAL VOC 2012 challenge. -Source: A Survey on Performance Metrics for Object-Detection Algorithms paper.
    - This is intuitive as well since this is object detection problem, therefore a gt bbox should only have 1 corresponding pred bbox. If you do not punish the additional pred bboxes, then it will inflate the TP value!

**Algorithm**

- Sort all pred bboxes by their confidence scores, in here we have 12 predicted bboxes with 9 gt bboxes.
- At the first row and also the first *confidence threshold*, we denote using our rules to check if this pred bbox $e$ is a TP or FP. 
    - It is TP and so our precision is 1 and recall is $1/9 = 0.11$. 
    - Note $TP + FN = 9$ always for object detection since there are 9 gt bboxes. 
    - Now we have a tuple of $(precision, recall)$ for this particular confidence threshold.
    - You now can think of our usual classification problem, where we use probability logits as threshold to get each pair of precision recall.
    - Another way to think is: if given the stringent confidence score (since sorted as highest), let's see how our model perform. Let us have this thinking.
    - Also, take note that in the first row, we are also asking: ok so we can only predict 1 bbox, how good is our model prediction?

- At the second row, we see that $a$ is a TP still.
    - precision is 1 and recall 0.22;
    - Note carefully that precision here is **cumulative** and so is **recall**, that is precision is calculated as 2/2 and recall as 2/9.
    - We have a new tuple of $(precision, recall)$ for this threshold of 0.98.
    - We can now think again: if given the confidence at 0.98 cutoff, how does our model perform? *note carefully that at 0.98 cutoff, we are including the first row's 0.99*.
    - We can now think again: if the model can predict 2 bboxes, how good is our predictions?
    
- We continue this process:

> at each confidence level (threshold), we ask what is the precision-recall score of the predictions of **all bounding boxes at a specific IOU** while discarding off those below the threshold, then average them over all thresholds? That's MAP